Title: Popular Classification Algorithms

Support Vector Machines (SVM)

Task 1: Identify handwriting on checks and classify each letter.

In [4]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim

# Step 1: Define transforms (normalize images)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))  # mean & std for EMNIST
])

# Step 2: Load EMNIST Letters dataset (train and test)
trainset = torchvision.datasets.EMNIST(root='./data', split='letters', train=True, download=True, transform=transform)
testset = torchvision.datasets.EMNIST(root='./data', split='letters', train=False, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

# Step 3: Define CNN model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.fc1 = nn.Linear(9216, 128)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 27)  # 26 letters + 1 (EMNIST letters labels start from 1)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

# Step 4: Instantiate model, loss, optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Step 5: Training loop (1 epoch example)
model.train()
for epoch in range(1):
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels - 1)  # EMNIST letters label starts from 1, so subtract 1
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} done")

# Step 6: Simple evaluation on test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == (labels - 1)).sum().item()

print(f"Test accuracy: {100 * correct / total:.2f}%")


ModuleNotFoundError: No module named 'torchvision'


Task 2: Detect gender of a speaker based on voice data.

In [5]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score

# Function to extract MFCC features from audio file
def extract_features(file_path):
    audio, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
    return np.mean(mfccs.T, axis=0)  # mean over time frames

# Prepare dataset (folder structure: dataset/male/*.wav and dataset/female/*.wav)
dataset_path = "dataset"
features = []
labels = []

for gender in ['male', 'female']:
    folder = os.path.join(dataset_path, gender)
    for file in os.listdir(folder):
        if file.endswith('.wav'):
            filepath = os.path.join(folder, file)
            features.append(extract_features(filepath))
            labels.append(gender)

# Convert to numpy arrays
X = np.array(features)
y = np.array(labels)

# Encode labels (male=1, female=0)
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Train Random Forest classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=le.classes_))


ModuleNotFoundError: No module named 'librosa'

Task 3: Classify email topics based on content.

In [4]:
# Write your code here